## Changing logits based on desired alliteration

We want to generate a poem where as many words as possible start with a desired letter

In [ ]:
#%pip install --upgrade --quiet transformers torch fbgemm-gpu accelerate

In [2]:
# CHANGE this to the Llama model for which you have applied for access via Hugging Face
# See: https://www.llama.com/docs/getting-the-models/hugging-face/
MODEL_ID = "meta-llama/Llama-3.2-3B-Instruct"

import os
from dotenv import load_dotenv
load_dotenv("../keys.env")
assert os.environ["HF_TOKEN"][:2] == "hf",\
       "Please sign up for access to the specific Llama model via HuggingFace and provide access token in keys.env file"

## Zero-shot generation

Without any logits processing

In [3]:
from transformers import pipeline

pipe = pipeline(
    task="text-generation", 
    model=MODEL_ID,
    use_fast=True,
    kwargs={
        "return_full_text": False,
    },
    model_kwargs={}
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


In [5]:
def generate_poem(animal: str) -> str:
    system_prompt = f"""
        You are writing nursery rhymes about animals for a children's book.
        Each poem should be 3-5 lines long.
        Return only the poem, without any introduction or preamble.
    """
    user_prompt = f"""
        Write a poem about a {animal}.
    """

    input_message = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}   
    ]
    
    results = pipe(input_message, 
                   max_new_tokens=256)
    return results[0]['generated_text'][-1]['content'].strip()

poem = generate_poem("donkey")
print(poem)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Hee-haw, the donkey sings,
A loud voice that joyfully brings,
A smile to the heart of a child.


Result:
```
Hee-haw, a donkey so bright,
Carrying loads with all his might,
His soft fur and gentle eyes shine,
A friendly friend, always on my mind.
```
Well shine & mind is not the perfect rhyme,
but still pretty good ... but if we ask for the poem about donkeys to
also be alliterative?

In [14]:
def generate_alliterative_poem(animal: str) -> str:
    system_prompt = f"""
        You are writing alliterative nursery rhymes about animals for a children's book.
        Each poem should be 3-5 lines long.
        Return only the poem, without any introduction or preamble.
    """
    
    # alliterate on the first letter of the animal. So, donkey would be D
    user_prompt = f"""
        Write a poem about a {animal} that contains alliterations involving the letter {animal.upper()[0]}
    """

    input_message = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}   
    ]
    
    results = pipe(input_message, 
                   max_new_tokens=256)
    return results[0]['generated_text'][-1]['content'].strip()

poem = generate_alliterative_poem("donkey")
print(poem)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Dainty donkeys dance down dusty roads,
Delightful droplets dripping from their noses' folds.
Doting dads delight in their darling donkey's doze.


Result:
```
Dainty donkeys dance down dusty roads,
Delightful droplets dripping from their noses' folds.
Doting dads delight in their darling donkey's doze.
```
The poem above is not great. By trying to match the style, the quality has gone way down.
Does a donkey with snot dripping from its nostrils delightful?

## Use logits processing to enhance the alliteration

We'll use the poem prompt, but use logits processing to prefer words that start with the desired letter

In [4]:
import torch
import numpy as np
from transformers.generation.logits_process import (
    LogitsProcessor,
    LOGITS_PROCESSOR_INPUTS_DOCSTRING,
)
from transformers.utils import add_start_docstrings

class AlliterativeLogitsProcessor(LogitsProcessor):
    def __init__(self, tokenizer, start_letter):
        self.tokenizer = tokenizer
        self.start_letter = start_letter
      
    @add_start_docstrings(LOGITS_PROCESSOR_INPUTS_DOCSTRING)
    def __call__(
        self, input_ids: torch.LongTensor, input_logits: torch.FloatTensor
    ) -> torch.FloatTensor:
        output_logits = input_logits.clone()
            
        num_matches = [0] * len(input_ids)
        for idx, seq in enumerate(input_ids):
            # decode the sequence
            decoded = self.tokenizer.decode(seq)
            # count the number of words that start with desired letter
            num_matches[idx] = np.sum([1 for word in decoded.split() if word[0] == self.start_letter])
        max_matches = np.max(num_matches)
          
        # logits goes from -inf to zero.  Mask out the non-max sequences; torch doesn't like it to be -np.inf
        for idx in range(len(input_ids)):
            if num_matches[idx] != max_matches:
                output_logits[idx] = -10000
                  
        return output_logits

In [4]:
def generate_alliterative_poem_v2(animal: str) -> str:
    system_prompt = f"""
        You are writing nursery rhymes about animals for a children's book.
        Each poem should be 3-5 lines long.
        Return only the poem, without any introduction or preamble.
    """
    user_prompt = f"""
        Write a poem about a {animal}.
    """

    input_message = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}   
    ]
    
    # alliterate on the first letter of the animal. So, donkey would be D
    grammar_processor = AlliterativeLogitsProcessor(pipe.tokenizer, animal[0])
    
    results = pipe(input_message, 
                   max_new_tokens=256,
                   do_sample=True,
                   temperature=0.8,
                   num_beams=10,
                   use_cache=True, # default is True
                   logits_processor=[grammar_processor])
    return results[0]['generated_text'][-1]['content'].strip()

poem = generate_alliterative_poem_v2("donkey")
print(poem)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Little donkey, ears so bright,
Hee-hawing loud through day's delight,
He trots along with gentle pace,
A friendly friend in a sunny place.


Result:
```
Little donkey, ears so bright,
Hee-hawing loud through day's delight,
He trots along with gentle pace,
A friendly friend in a sunny place.
```
Has 3 ds

## Combine prompting and sequence selection

Enhance the prompt but make it clear we want it to be readable,
but use logits processing to prefer words that start with the desired letter

In [13]:
def generate_alliterative_poem_v3(animal: str) -> str:
    start_letter = animal[0]
    
    system_prompt = f"""
        You are writing nursery rhymes about animals for a children's book.
        Each poem should be 3-5 lines long. The poem must be readable and suitable for children.
        Return only the poem, without any introduction or preamble.
    """
    user_prompt = f"""
        Write a poem about a {animal} that has a few alliterations involving {start_letter}.
        Do not overdo alliteration, and emphasize readability.
    """

    input_message = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}   
    ]
    
    # alliterate on the first letter of the animal. So, donkey would be D
    grammar_processor = AlliterativeLogitsProcessor(pipe.tokenizer, start_letter)
    
    results = pipe(input_message, 
                   max_new_tokens=256,
                   do_sample=True,
                   temperature=0.8,
                   num_beams=10,
                   use_cache=True, # default is True
                   logits_processor=[grammar_processor])
    return results[0]['generated_text'][-1]['content'].strip()

poem = generate_alliterative_poem_v3("donkey")
print(poem)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Down the dusty desert, donkey did stray
Dreaming of delicious dates to devour each day
Dainty donkey danced down the desert way


Result:
```
Down the dusty desert, donkey did stray
Dreaming of delicious dates to devour each day
Dainty donkey danced down the desert way
```
It's still a readable poem, but we have picked the one with the most ds.